In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/skin-cancer7f/tensorflow2/default/1/Skin Cancer7.keras
/kaggle/input/skin-cancer-malignant-vs-benign/data/test/benign/1269.jpg
/kaggle/input/skin-cancer-malignant-vs-benign/data/test/benign/623.jpg
/kaggle/input/skin-cancer-malignant-vs-benign/data/test/benign/820.jpg
/kaggle/input/skin-cancer-malignant-vs-benign/data/test/benign/1354.jpg
/kaggle/input/skin-cancer-malignant-vs-benign/data/test/benign/45.jpg
/kaggle/input/skin-cancer-malignant-vs-benign/data/test/benign/369.jpg
/kaggle/input/skin-cancer-malignant-vs-benign/data/test/benign/1711.jpg
/kaggle/input/skin-cancer-malignant-vs-benign/data/test/benign/1173.jpg
/kaggle/input/skin-cancer-malignant-vs-benign/data/test/benign/1488.jpg
/kaggle/input/skin-cancer-malignant-vs-benign/data/test/benign/436.jpg
/kaggle/input/skin-cancer-malignant-vs-benign/data/test/benign/1409.jpg
/kaggle/input/skin-cancer-malignant-vs-benign/data/test/benign/71.jpg
/kaggle/input/skin-cancer-malignant-vs-benign/data/test/benign/1055.jpg
/ka

In [2]:
!pip uninstall -y jax jaxlib
!pip install --upgrade --force-reinstall tensorflow

Found existing installation: jax 0.5.2
Uninstalling jax-0.5.2:
  Successfully uninstalled jax-0.5.2
Found existing installation: jaxlib 0.5.1
Uninstalling jaxlib-0.5.1:
  Successfully uninstalled jaxlib-0.5.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 122.7 kB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 72.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 72.2 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 63.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 74.2 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from collections import Counter
import math
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import os
from PIL import Image
import random
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras import layers, models
import tensorflow as tf 
from tensorflow.keras.regularizers import L2
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.metrics import confusion_matrix,precision_score, recall_score
from tensorflow.keras.applications.mobilenet import preprocess_input


In [3]:
train_data=os.listdir("/kaggle/input/skin-cancer-malignant-vs-benign/train")
test_data=os.listdir("/kaggle/input/skin-cancer-malignant-vs-benign/test")

cancer_types_train_data=[]
train_labels=[]
for item in train_data:
  all_categ_train=os.listdir('/kaggle/input/skin-cancer-malignant-vs-benign/train'+'/'+item)
  filenames=[j for j in os.listdir('/kaggle/input/skin-cancer-malignant-vs-benign/train'+'/'+item) if j.endswith('.jpg')]
  for f in filenames:
    image_path = os.path.join('/kaggle/input/skin-cancer-malignant-vs-benign/train'+'/'+item, f)
    cancer_types_train_data.append((item,image_path))
    train_labels.append(item)
print(cancer_types_train_data)

[('benign', '/kaggle/input/skin-cancer-malignant-vs-benign/train/benign/764.jpg'), ('benign', '/kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1700.jpg'), ('benign', '/kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1786.jpg'), ('benign', '/kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1075.jpg'), ('benign', '/kaggle/input/skin-cancer-malignant-vs-benign/train/benign/771.jpg'), ('benign', '/kaggle/input/skin-cancer-malignant-vs-benign/train/benign/208.jpg'), ('benign', '/kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1789.jpg'), ('benign', '/kaggle/input/skin-cancer-malignant-vs-benign/train/benign/473.jpg'), ('benign', '/kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1792.jpg'), ('benign', '/kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1411.jpg'), ('benign', '/kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1031.jpg'), ('benign', '/kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1749.jpg'), ('benig

In [4]:
cancer_types_test_data=[]
test_labels=[]
for item in test_data:
  all_categ_test=os.listdir(str('/kaggle/input/skin-cancer-malignant-vs-benign/test'+'/'+item))
  print(all_categ_test[:])
  filenames=[j for j in os.listdir('/kaggle/input/skin-cancer-malignant-vs-benign/test'+'/'+item) if j.endswith('.jpg')]
  for f in filenames:
    image_path = os.path.join('/kaggle/input/skin-cancer-malignant-vs-benign/test'+'/'+item, f)
    cancer_types_test_data.append((item,image_path))
    test_labels.append(item)
print(cancer_types_test_data)

['1269.jpg', '623.jpg', '820.jpg', '1354.jpg', '45.jpg', '369.jpg', '1711.jpg', '1173.jpg', '1488.jpg', '436.jpg', '1409.jpg', '71.jpg', '1055.jpg', '215.jpg', '185.jpg', '1025.jpg', '1343.jpg', '1579.jpg', '717.jpg', '1473.jpg', '253.jpg', '1166.jpg', '626.jpg', '1443.jpg', '5.jpg', '1559.jpg', '824.jpg', '1376.jpg', '1297.jpg', '8.jpg', '892.jpg', '1655.jpg', '84.jpg', '751.jpg', '944.jpg', '237.jpg', '273.jpg', '289.jpg', '1125.jpg', '768.jpg', '1134.jpg', '1243.jpg', '1499.jpg', '668.jpg', '97.jpg', '806.jpg', '1401.jpg', '211.jpg', '1648.jpg', '291.jpg', '1019.jpg', '10.jpg', '54.jpg', '251.jpg', '916.jpg', '1183.jpg', '558.jpg', '1118.jpg', '998.jpg', '1592.jpg', '1018.jpg', '61.jpg', '1191.jpg', '1359.jpg', '427.jpg', '1200.jpg', '819.jpg', '379.jpg', '1288.jpg', '313.jpg', '1310.jpg', '1080.jpg', '98.jpg', '527.jpg', '580.jpg', '1710.jpg', '1471.jpg', '256.jpg', '1034.jpg', '559.jpg', '1634.jpg', '57.jpg', '1146.jpg', '894.jpg', '690.jpg', '664.jpg', '388.jpg', '1044.jpg', '174

In [5]:
print(len(cancer_types_test_data))

660


In [6]:
test_labels.extend(train_labels)
df_labels=test_labels
print(len(df_labels))

3297


In [7]:
cancer_df_train=pd.DataFrame(data=cancer_types_train_data,columns=['cancer_type','image_path'])
cancer_df_train

,cancer_type,image_path
0,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
1,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
2,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
3,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
4,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
...,...,...
2632,malignant,/kaggle/input/skin-cancer-malignant-vs-benign/...
2633,malignant,/kaggle/input/skin-cancer-malignant-vs-benign/...
2634,malignant,/kaggle/input/skin-cancer-malignant-vs-benign/...
2635,malignant,/kaggle/input/skin-cancer-malignant-vs-benign/...


In [8]:
cancer_df_test=pd.DataFrame(data=cancer_types_test_data,columns=['cancer_type','image_path'])
cancer_df_test

,cancer_type,image_path
0,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
1,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
2,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
3,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
4,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
...,...,...
655,malignant,/kaggle/input/skin-cancer-malignant-vs-benign/...
656,malignant,/kaggle/input/skin-cancer-malignant-vs-benign/...
657,malignant,/kaggle/input/skin-cancer-malignant-vs-benign/...
658,malignant,/kaggle/input/skin-cancer-malignant-vs-benign/...


In [9]:
df=pd.concat([cancer_df_train,cancer_df_test])
df

,cancer_type,image_path
0,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
1,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
2,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
3,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
4,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
...,...,...
655,malignant,/kaggle/input/skin-cancer-malignant-vs-benign/...
656,malignant,/kaggle/input/skin-cancer-malignant-vs-benign/...
657,malignant,/kaggle/input/skin-cancer-malignant-vs-benign/...
658,malignant,/kaggle/input/skin-cancer-malignant-vs-benign/...


In [ ]:


# STEP 1: Simulate Imbalanced Dataset (benign > malignant)
malignant_df = df[df['cancer_type'] == 'malignant']
benign_df = df[df['cancer_type'] == 'benign']
df_imbalanced = pd.concat([benign_df, malignant_df], ignore_index=True)

print(f"Simulated Imbalanced dataset length: {len(df_imbalanced)}")
print(f"Simulated class distribution: {Counter(df_imbalanced['cancer_type'])}")

# Replace df with the imbalanced version
df = df_imbalanced

# STEP 2: Create ImageDataGenerator for augmentation
augmentor = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# STEP 3: Count classes
labels_counter = Counter(df['cancer_type'])
num_benign = labels_counter.get('benign', 0)
num_malignant = labels_counter.get('malignant', 0)

print(f"Before Augmentation: {num_benign} benign vs {num_malignant} malignant")

# STEP 4: Compute needed augmentations
additional_malignant_needed = num_benign - num_malignant

if additional_malignant_needed > 0:
    malignant_images_paths = df[df['cancer_type'] == 'malignant']['image_path'].tolist()
    augment_copies_per_image = math.ceil(additional_malignant_needed / len(malignant_images_paths))

    print(f"Augmenting ~{augment_copies_per_image} copies per malignant image")

    augmented_images = []
    save_dir = '/kaggle/working/augmented/malignant'
    os.makedirs(save_dir, exist_ok=True)

    count = 0
    for img_idx, img_path in enumerate(malignant_images_paths):
        img = load_img(img_path)
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)

        aug_iter = augmentor.flow(img_array, batch_size=1)

        for _ in range(augment_copies_per_image):
            aug_img_array = next(aug_iter)[0].astype('uint8')
            img_pil = Image.fromarray(aug_img_array)

            save_path = os.path.join(save_dir, f'aug_malignant_{count}.jpg')
            img_pil.save(save_path)

            augmented_images.append({'cancer_type': 'malignant', 'image_path': save_path})
            count += 1

            if count >= additional_malignant_needed:
                break
        if count >= additional_malignant_needed:
            break

    print(f"Number of augmented malignant images generated: {len(augmented_images)}")

    df_augmented = pd.DataFrame(augmented_images)
    df = pd.concat([df, df_augmented], ignore_index=True)

else:
    print("No augmentation needed, classes are already balanced.")

# STEP 5: Final Summary
print(f"Final dataset length: {len(df)}")
print(f"Final class distribution: {Counter(df['cancer_type'])}")


2025-06-18 08:32:32.766690: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750235552.789002      80 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750235552.795819      80 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750235552.814069      80 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750235552.814095      80 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750235552.814098      80 computation_placer.cc:177] computation placer alr

Simulated Imbalanced dataset length: 3297
Simulated class distribution: Counter({'benign': 1800, 'malignant': 1497})
Before Augmentation: 1800 benign vs 1497 malignant
Augmenting ~1 copies per malignant image
Number of augmented malignant images generated: 303
Final dataset length: 3600
Final class distribution: Counter({'benign': 1800, 'malignant': 1800})


In [11]:
df

,cancer_type,image_path
0,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
1,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
2,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
3,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
4,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
...,...,...
3595,malignant,/kaggle/working/augmented/malignant/aug_malign...
3596,malignant,/kaggle/working/augmented/malignant/aug_malign...
3597,malignant,/kaggle/working/augmented/malignant/aug_malign...
3598,malignant,/kaggle/working/augmented/malignant/aug_malign...


In [12]:
df_benign=df[df['cancer_type']=='benign']
df_benign

,cancer_type,image_path
0,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
1,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
2,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
3,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
4,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
...,...,...
1795,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
1796,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
1797,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
1798,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...


In [13]:
df_malignant=df[df['cancer_type']=='malignant']
df_malignant

,cancer_type,image_path
1800,malignant,/kaggle/input/skin-cancer-malignant-vs-benign/...
1801,malignant,/kaggle/input/skin-cancer-malignant-vs-benign/...
1802,malignant,/kaggle/input/skin-cancer-malignant-vs-benign/...
1803,malignant,/kaggle/input/skin-cancer-malignant-vs-benign/...
1804,malignant,/kaggle/input/skin-cancer-malignant-vs-benign/...
...,...,...
3595,malignant,/kaggle/working/augmented/malignant/aug_malign...
3596,malignant,/kaggle/working/augmented/malignant/aug_malign...
3597,malignant,/kaggle/working/augmented/malignant/aug_malign...
3598,malignant,/kaggle/working/augmented/malignant/aug_malign...


In [14]:
df_train_benign=df_benign[:1530]
df_train_benign

,cancer_type,image_path
0,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
1,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
2,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
3,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
4,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
...,...,...
1525,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
1526,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
1527,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
1528,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...


In [21]:
df_train_malignant=df_malignant[:1530]
df_train_malignant

,cancer_type,image_path
1800,malignant,/kaggle/input/skin-cancer-malignant-vs-benign/...
1801,malignant,/kaggle/input/skin-cancer-malignant-vs-benign/...
1802,malignant,/kaggle/input/skin-cancer-malignant-vs-benign/...
1803,malignant,/kaggle/input/skin-cancer-malignant-vs-benign/...
1804,malignant,/kaggle/input/skin-cancer-malignant-vs-benign/...
...,...,...
3325,malignant,/kaggle/working/augmented/malignant/aug_malign...
3326,malignant,/kaggle/working/augmented/malignant/aug_malign...
3327,malignant,/kaggle/working/augmented/malignant/aug_malign...
3328,malignant,/kaggle/working/augmented/malignant/aug_malign...


In [22]:
df_train=pd.concat([df_train_benign,df_train_malignant])
df_train= df_train.sample(frac=1).reset_index(drop=True)
print(df_train)

     cancer_type                                         image_path
0         benign  /kaggle/input/skin-cancer-malignant-vs-benign/...
1      malignant  /kaggle/input/skin-cancer-malignant-vs-benign/...
2         benign  /kaggle/input/skin-cancer-malignant-vs-benign/...
3         benign  /kaggle/input/skin-cancer-malignant-vs-benign/...
4         benign  /kaggle/input/skin-cancer-malignant-vs-benign/...
...          ...                                                ...
3055   malignant  /kaggle/input/skin-cancer-malignant-vs-benign/...
3056      benign  /kaggle/input/skin-cancer-malignant-vs-benign/...
3057      benign  /kaggle/input/skin-cancer-malignant-vs-benign/...
3058      benign  /kaggle/input/skin-cancer-malignant-vs-benign/...
3059   malignant  /kaggle/input/skin-cancer-malignant-vs-benign/...

[3060 rows x 2 columns]


In [23]:
df_test_benign=df_benign[1530:]
df_test_benign

,cancer_type,image_path
1530,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
1531,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
1532,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
1533,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
1534,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
...,...,...
1795,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
1796,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
1797,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...
1798,benign,/kaggle/input/skin-cancer-malignant-vs-benign/...


In [24]:
df_test_malignant=df_malignant[1530:]
df_test_malignant

,cancer_type,image_path
3330,malignant,/kaggle/working/augmented/malignant/aug_malign...
3331,malignant,/kaggle/working/augmented/malignant/aug_malign...
3332,malignant,/kaggle/working/augmented/malignant/aug_malign...
3333,malignant,/kaggle/working/augmented/malignant/aug_malign...
3334,malignant,/kaggle/working/augmented/malignant/aug_malign...
...,...,...
3595,malignant,/kaggle/working/augmented/malignant/aug_malign...
3596,malignant,/kaggle/working/augmented/malignant/aug_malign...
3597,malignant,/kaggle/working/augmented/malignant/aug_malign...
3598,malignant,/kaggle/working/augmented/malignant/aug_malign...


In [25]:
df_test=pd.concat([df_test_benign,df_test_malignant])
df_test= df_test.sample(frac=1).reset_index(drop=True)
print(df_test)

    cancer_type                                         image_path
0        benign  /kaggle/input/skin-cancer-malignant-vs-benign/...
1        benign  /kaggle/input/skin-cancer-malignant-vs-benign/...
2     malignant  /kaggle/working/augmented/malignant/aug_malign...
3        benign  /kaggle/input/skin-cancer-malignant-vs-benign/...
4        benign  /kaggle/input/skin-cancer-malignant-vs-benign/...
..          ...                                                ...
535      benign  /kaggle/input/skin-cancer-malignant-vs-benign/...
536   malignant  /kaggle/working/augmented/malignant/aug_malign...
537      benign  /kaggle/input/skin-cancer-malignant-vs-benign/...
538   malignant  /kaggle/working/augmented/malignant/aug_malign...
539      benign  /kaggle/input/skin-cancer-malignant-vs-benign/...

[540 rows x 2 columns]


In [ ]:

# Augmentation for the training data
train_data_gen = ImageDataGenerator(
    rescale=1.0/255.0,                
    rotation_range=30,            
    width_shift_range=0.1,        
    height_shift_range=0.1,      
    zoom_range=0.3,              
    horizontal_flip=True,         
    fill_mode='constant',          
    cval=0

)

# Minimal augmentation for validation data (only rescaling)
val_data_gen = ImageDataGenerator(
    rescale=1.0/255.0,                
    validation_split=0.176, 

)

test_data_gen = ImageDataGenerator(rescale=1.0/255.0)

# Training generator with augmentations
train_generator = train_data_gen.flow_from_dataframe(
    dataframe=df_train,                  # DataFrame containing image paths and labels
    x_col='image_path',            # Column with image file paths
    y_col='cancer_type',             # Column with labels
    target_size=(224, 224),        # Resize images to 224x224
    class_mode='binary',      # Multi-class classification
    batch_size=16,                 # Number of images per batch
    subset='training',             # Use as training data
    shuffle=True  ,
    validation_split=0.176,  # Important!
    seed=1
)

# Validation generator with minimal augmentation
validation_generator = val_data_gen.flow_from_dataframe(
    dataframe=df_train,
    x_col='image_path',
    y_col='cancer_type',
    target_size=(224,224),
    class_mode='binary',
    batch_size=16,
    subset='validation',          # Use as validation data
    shuffle=False ,                # No shuffle for consistent validation
    seed=1
)



test_generator = test_data_gen.flow_from_dataframe(
    dataframe=df_test,               # A separate test dataframe
    x_col='image_path',
    y_col='cancer_type',            # You can also set y_col=None if doing inference only
    target_size=(224, 224),
    class_mode='binary',       # Or 'input' / None if not evaluating labels
    batch_size=16,
    shuffle=False                   # Important for matching predictions with inputs
)

print("Class Indices: ", train_generator.class_indices)
print("Class Indices: ", validation_generator.class_indices)
print("Class Indices: ", test_generator.class_indices)



Found 3060 validated image filenames belonging to 2 classes.
Found 538 validated image filenames belonging to 2 classes.
Found 540 validated image filenames belonging to 2 classes.
Class Indices:  {'benign': 0, 'malignant': 1}
Class Indices:  {'benign': 0, 'malignant': 1}
Class Indices:  {'benign': 0, 'malignant': 1}


In [ ]:


base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers[:-5]:
    layer.trainable = False    # Freeze the base model

model = Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu', kernel_regularizer=L2(0.08)),
    layers.Dropout(0.3),
    layers.Dense(512, activation='relu', kernel_regularizer=L2(0.08)),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')  # Adjust the number of classes
])


model.summary()


I0000 00:00:1750235633.910951      80 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenet_1.00_224 (Functional)      │ (None, 7, 7, 1024)          │       3,228,864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1024)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1024)                │       1,049,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │             513 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,803,777 (18.32 MB)

 Trainable params: 2,627,585 (10.02 MB)

 Non-trainable params: 2,176,192 (8.30 MB)

In [ ]:

optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy','recall','precision'])

In [ ]:



# Set EarlyStopping callback with patience=5


# Enhanced Callbacks

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True,
    start_from_epoch=0
)
# Add callbacks to training

# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=50,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    shuffle=True,
    callbacks=[early_stopping]
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50


I0000 00:00:1750235648.665497     175 service.cc:152] XLA service 0x7e8bb000d690 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1750235648.665527     175 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1750235649.570310     175 cuda_dnn.cc:529] Loaded cuDNN version 90300


  1/191 ━━━━━━━━━━━━━━━━━━━━ 29:44 9s/step - accuracy: 0.5625 - loss: 137.2679 - precision: 0.6000 - recall: 0.9000

I0000 00:00:1750235653.810565     175 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


191/191 ━━━━━━━━━━━━━━━━━━━━ 58s 256ms/step - accuracy: 0.7121 - loss: 112.0537 - precision: 0.7087 - recall: 0.7456 - val_accuracy: 0.5852 - val_loss: 55.2525 - val_precision: 0.9661 - val_recall: 0.2080 - learning_rate: 1.0000e-04
Epoch 2/50
  1/191 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.7500 - loss: 55.0270 - precision: 0.7500 - recall: 0.7500

/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/callback_list.py:145: UserWarning: Learning rate reduction is conditioned on metric `val_percision` which is not available. Available metrics are: accuracy,loss,precision,recall,val_accuracy,val_loss,val_precision,val_recall,learning_rate.
  callback.on_epoch_end(epoch, logs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7500 - loss: 55.0270 - precision: 0.7500 - recall: 0.7500 - val_accuracy: 0.5777 - val_loss: 55.0084 - val_precision: 0.9636 - val_recall: 0.1934 - learning_rate: 1.0000e-04
Epoch 3/50
191/191 ━━━━━━━━━━━━━━━━━━━━ 33s 175ms/step - accuracy: 0.8098 - loss: 43.6736 - precision: 0.8160 - recall: 0.8024 - val_accuracy: 0.7917 - val_loss: 19.8828 - val_precision: 0.9184 - val_recall: 0.6569 - learning_rate: 1.0000e-04
Epoch 4/50
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8125 - loss: 19.7902 - precision: 0.8750 - recall: 0.7778 - val_accuracy: 0.7992 - val_loss: 19.7633 - val_precision: 0.9118 - val_recall: 0.6788 - learning_rate: 1.0000e-04
Epoch 5/50
191/191 ━━━━━━━━━━━━━━━━━━━━ 34s 176ms/step - accuracy: 0.8477 - loss: 15.4935 - precision: 0.8267 - recall: 0.8707 - val_accuracy: 0.8087 - val_loss: 6.7341 - val_precision: 0.9303 - val_recall: 0.6825 - learning_rate: 1.0000e-04
Epoch 6/50
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms

In [30]:
print("Evaluating model...")
train_loss, train_accuracy,train_recall,train_precision = model.evaluate(train_generator, steps=train_generator.samples // train_generator.batch_size)
print(f"training loss: {train_loss * 100:.2f}%")
print(f"training Accuracy: {train_accuracy * 100:.2f}%")
print(f"training precision: {train_precision * 100:.2f}%")
print(f"training recall: {train_recall  * 100:.2f}%")


print("Evaluating model...")
val_loss, val_accuracy,val_recall,val_precision = model.evaluate(validation_generator, steps=validation_generator.samples // validation_generator.batch_size)
print(f"val loss: {val_loss * 100:.2f}%")
print(f"val Accuracy: {val_accuracy * 100:.2f}%")
print(f"val precision: {val_precision * 100:.2f}%")
print(f"val recall: {val_recall  * 100:.2f}%")


test_loss, test_accuracy ,test_recall,test_precision= model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f"test_Accuracy: {test_accuracy * 100:.2f}%")
print(f"test_ loss: {test_loss * 100:.2f}%")
print(f"test_precision {test_precision * 100:.2f}%")
print(f"test_recall: {test_recall * 100:.2f}%")



Evaluating model...
191/191 ━━━━━━━━━━━━━━━━━━━━ 32s 169ms/step - accuracy: 0.8630 - loss: 0.3878 - precision: 0.8234 - recall: 0.9111
training loss: 37.19%
training Accuracy: 87.66%
training precision: 85.07%
training recall: 91.36%
Evaluating model...
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - accuracy: 0.8951 - loss: 0.3441 - precision: 0.8964 - recall: 0.8932
val loss: 35.99%
val Accuracy: 88.64%
val precision: 89.63%
val recall: 88.32%
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step - accuracy: 0.9307 - loss: 0.2701 - precision: 0.8764 - recall: 1.0000
test_Accuracy: 93.37%
test_ loss: 25.60%
test_precision 88.26%
test_recall: 100.00%


In [45]:
for layer in base_model.layers[-50:]:
    layer.trainable = True    #Unfreeze the base model


# Lower learning rate for fine-tuning
model.compile(
    optimizer = Adam(learning_rate=0.0001),
    loss="binary_crossentropy",
    metrics=['accuracy','recall','precision']
)


In [46]:
reduce_lr= ReduceLROnPlateau(
    monitor='val_precision',
    factor=0.5,
    patience=2,
    min_lr=1e-8,
    verbose=1
)


early_stopping = EarlyStopping(
    monitor='val_precision',
    patience=5,
    restore_best_weights=True,
    start_from_epoch=0
)
# Continue training
fine_tune_epochs = 30
history_fine = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=fine_tune_epochs,
    steps_per_epoch=train_generator.samples // 16,
    validation_steps=validation_generator.samples // 16,
    shuffle=True,
    callbacks=[reduce_lr,early_stopping]
)


Epoch 1/30
191/191 ━━━━━━━━━━━━━━━━━━━━ 53s 199ms/step - accuracy: 0.9552 - loss: 0.1679 - precision: 0.9538 - recall: 0.9580 - val_accuracy: 0.9848 - val_loss: 0.1057 - val_precision: 0.9716 - val_recall: 1.0000 - learning_rate: 1.0000e-04
Epoch 2/30
191/191 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 1.0000 - loss: 0.0989 - precision: 1.0000 - recall: 1.0000 - val_accuracy: 0.9848 - val_loss: 0.1044 - val_precision: 0.9716 - val_recall: 1.0000 - learning_rate: 1.0000e-04
Epoch 3/30
191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step - accuracy: 0.9544 - loss: 0.1671 - precision: 0.9603 - recall: 0.9499
Epoch 3: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
191/191 ━━━━━━━━━━━━━━━━━━━━ 34s 179ms/step - accuracy: 0.9544 - loss: 0.1671 - precision: 0.9603 - recall: 0.9499 - val_accuracy: 0.9886 - val_loss: 0.0873 - val_precision: 0.9891 - val_recall: 0.9891 - learning_rate: 1.0000e-04
Epoch 4/30
191/191 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 1.0000 - loss: 0.0980 - pr

In [47]:
print("Evaluating model...")
train_loss, train_accuracy,train_recall,train_precision = model.evaluate(train_generator, steps=train_generator.samples // train_generator.batch_size)
print(f"training loss: {train_loss * 100:.2f}%")
print(f"training Accuracy: {train_accuracy * 100:.2f}%")
print(f"training precision: {train_precision * 100:.2f}%")
print(f"training recall: {train_recall  * 100:.2f}%")


print("Evaluating model...")
val_loss, val_accuracy,val_recall,val_precision = model.evaluate(validation_generator, steps=validation_generator.samples // validation_generator.batch_size)
print(f"val loss: {val_loss * 100:.2f}%")
print(f"val Accuracy: {val_accuracy * 100:.2f}%")
print(f"val precision: {val_precision * 100:.2f}%")
print(f"val recall: {val_recall  * 100:.2f}%")


test_loss, test_accuracy ,test_recall,test_precision= model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f"test_Accuracy: {test_accuracy * 100:.2f}%")
print(f"test_ loss: {test_loss * 100:.2f}%")
print(f"test_precision {test_precision * 100:.2f}%")
print(f"test_recall: {test_recall * 100:.2f}%")



Evaluating model...
191/191 ━━━━━━━━━━━━━━━━━━━━ 33s 172ms/step - accuracy: 0.9857 - loss: 0.0926 - precision: 0.9976 - recall: 0.9740
training loss: 9.36%
training Accuracy: 98.46%
training precision: 99.60%
training recall: 97.32%
Evaluating model...
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.9879 - loss: 0.0853 - precision: 1.0000 - recall: 0.9759
val loss: 8.54%
val Accuracy: 98.86%
val precision: 100.00%
val recall: 97.81%
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.9566 - loss: 0.1746 - precision: 0.9187 - recall: 1.0000
test_Accuracy: 96.40%
test_ loss: 14.33%
test_precision 93.26%
test_recall: 100.00%


In [ ]:
# Get predictions on the whole test set
preds = model.predict(test_generator, verbose=1)
print(preds)
y_pred= (preds >= 0.965).astype(int)
print(y_pred)
y_true = test_generator.classes  # works if you're using flow_from_directory or flow_from_dataframe

34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step
[[6.57037180e-03]
 [5.59417829e-02]
 [9.83021617e-01]
 [1.29334955e-03]
 [3.48072354e-06]
 [9.96911287e-01]
 [9.90246058e-01]
 [5.13705891e-03]
 [9.96292293e-01]
 [1.92579981e-02]
 [1.63976941e-02]
 [3.11977882e-03]
 [9.83800948e-01]
 [2.23842859e-02]
 [9.99452174e-01]
 [3.04019690e-01]
 [6.76990867e-01]
 [9.91537809e-01]
 [9.75414217e-01]
 [9.91313994e-01]
 [9.92846787e-01]
 [3.12324855e-02]
 [9.89144623e-01]
 [1.38246559e-03]
 [9.98448730e-01]
 [5.96462429e-01]
 [9.86425161e-01]
 [4.96025756e-02]
 [9.97626483e-01]
 [9.88730848e-01]
 [9.63874221e-01]
 [3.32707004e-03]
 [9.92228687e-01]
 [3.27136591e-02]
 [6.18782686e-03]
 [9.95633066e-01]
 [3.46258079e-04]
 [8.27797968e-03]
 [9.28423405e-01]
 [9.73082781e-01]
 [9.94296253e-01]
 [9.97529566e-01]
 [9.93524075e-01]
 [9.90111172e-01]
 [5.46329692e-02]
 [9.92847025e-01]
 [9.98340607e-01]
 [7.05400884e-01]
 [9.96268272e-01]
 [9.99507785e-01]
 [3.28835304e-04]
 [9.97162521e-01]
 [9.24844325e-01]
 [9.82

In [ ]:

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[266   4]
 [ 13 257]]


In [73]:
print(df_test['cancer_type'].value_counts())

cancer_type
benign       270
malignant    270
Name: count, dtype: int64


In [51]:
model.save('Skin Cancer11.keras')

In [85]:
file_paths = [test_generator.directory + '/' + fname for fname in test_generator.filenames]
x=[]
for path in file_paths:
    x.append(path)
print(x)

['//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/1143.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/475.jpg', '//kaggle/working/augmented/malignant/aug_malignant_65.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/205.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/389.jpg', '//kaggle/working/augmented/malignant/aug_malignant_246.jpg', '//kaggle/working/augmented/malignant/aug_malignant_204.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/495.jpg', '//kaggle/working/augmented/malignant/aug_malignant_296.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/681.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/1074.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/612.jpg', '//kaggle/working/augmented/malignant/aug_malignant_166.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/532.jpg', '//kaggle/working/augmented/malignant/aug_malignant_92.jpg

In [79]:
print(len(file_paths))

538


In [ ]:

# Function to preprocess the image and predict
def preprocess_and_predict(image_path, model):
    img = image.load_img(image_path, target_size=(224, 224))  # Resize
    img_array = image.img_to_array(img)  # Convert to array
    img_array = preprocess_input(img_array)  # Normalize input for MobileNet
    img_array = tf.expand_dims(img_array, axis=0)  # Add batch dimension

    # Get model prediction
     # Get model prediction
    threshold = 0.965  # Increase the threshold to be more confident
    predictions = model.predict(img_array)
    predicted_label = 'Malignant' if predictions[0] >= threshold else 'Benign'
    return predicted_label

# Function to evaluate the model and calculate precision, recall, F1-score
def evaluate_model(test_image_urls, model):
    y_true = []  # List of true labels (Benign or Malignant)
    y_pred = []  # List of predicted labels (Benign or Malignant)
    
    # Loop through all images and make predictions
    for image_url in test_image_urls:
        # Extract the actual label from the URL (e.g., 'malignant' or 'benign')
        class_label_from_url = os.path.basename(os.path.dirname(image_url)).lower()
        
        # Get the predicted label
        predicted_label = preprocess_and_predict(image_url, model)
        predicted_label_normalized = predicted_label.lower()
        
        # Append the true label and predicted label to lists
        y_true.append(class_label_from_url)
        y_pred.append(predicted_label_normalized)
    
    # Calculate precision, recall, and F1-score
    precision = precision_score(y_true, y_pred, average='binary', pos_label='malignant')
    recall = recall_score(y_true, y_pred, average='binary', pos_label='malignant')
    f1 = f1_score(y_true, y_pred, average='binary', pos_label='malignant')

    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred, labels=['benign', 'malignant'])

    return precision, recall, f1, cm

# Example: List of image URLs (replace this with your actual test image paths)
test_image_urls =['//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/361.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/921.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/843.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/765.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1091.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/406.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1493.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/271.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/775.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/153.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1201.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1343.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/530.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/275.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1096.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1121.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/573.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/277.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/731.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1549.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/577.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1353.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/906.jpg', '//kaggle/working/augmented/malignant/aug_malignant_9.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1154.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/100.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/395.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1217.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/490.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/682.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1363.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1375.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1799.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/817.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/1173.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/944.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1210.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1316.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/288.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1252.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/67.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1484.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1119.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1152.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/189.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/736.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1233.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/283.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/914.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1271.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/733.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1227.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1213.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/656.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/525.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/99.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/555.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1298.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1358.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/794.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1287.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1781.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/501.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/745.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/752.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/709.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/765.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/92.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/411.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1347.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/880.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/523.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/14.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1129.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1457.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/992.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1246.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/509.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/664.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1685.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/975.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/169.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/205.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/647.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/562.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/276.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1444.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/484.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1451.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/252.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/941.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1453.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/950.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1461.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/146.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/513.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/104.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1040.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1618.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1434.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1049.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/247.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/258.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/683.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/697.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/907.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1435.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1237.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/678.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/782.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/134.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/28.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1237.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1583.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/753.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/51.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/358.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/375.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/993.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1286.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1075.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/699.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1631.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1273.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1221.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/623.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1762.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/645.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/727.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/243.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1342.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/445.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/19.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/621.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1633.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/243.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1577.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1482.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1222.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/794.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/522.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/403.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1142.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/444.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1442.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/161.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/661.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1358.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/41.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/617.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1062.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1090.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/629.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/130.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1260.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/12.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/1409.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1160.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1007.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/426.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1326.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/867.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1054.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/541.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/755.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/554.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/926.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1777.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/87.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/704.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1427.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/785.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/455.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/160.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/657.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/233.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1389.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1593.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/598.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1507.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/935.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1599.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1052.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1354.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/637.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/478.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/747.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1219.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1137.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/873.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1359.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/875.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1404.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1581.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/415.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/100.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/88.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/679.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1674.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1506.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1324.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1615.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/978.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1769.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/427.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/891.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1403.jpg', '//kaggle/working/augmented/malignant/aug_malignant_31.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/75.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1735.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/951.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/649.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1652.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/96.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/20.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1254.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/872.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/208.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/687.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1385.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/404.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/713.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/439.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/744.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1106.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/1288.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/152.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/860.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1346.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1393.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/536.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/770.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/308.jpg', '//kaggle/working/augmented/malignant/aug_malignant_28.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/131.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1722.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1488.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/825.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1626.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1184.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/227.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/896.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/57.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/930.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/133.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/821.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1439.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/32.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/931.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/652.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1691.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/1471.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/965.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/379.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1353.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1739.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/973.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/380.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/95.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/991.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1523.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1267.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/815.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/750.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1478.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1410.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/823.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1402.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/29.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/756.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/35.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/902.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/434.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/812.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1463.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/517.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/13.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1374.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1446.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/400.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/882.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/581.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1171.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/556.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/650.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/979.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/833.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1746.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1441.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/610.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/340.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/507.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1436.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/212.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1332.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/448.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/771.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/674.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1695.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/456.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/842.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/674.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1733.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/368.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/685.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/820.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1454.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1219.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1083.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1168.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1226.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/318.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/79.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/781.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1442.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/199.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/994.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/181.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1364.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1480.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/804.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/507.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1028.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/700.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/425.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/804.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/239.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/618.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/943.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/636.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1598.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1381.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1494.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/917.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/631.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/979.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/878.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1190.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/416.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1563.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/108.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/251.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/46.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/749.jpg', '//kaggle/working/augmented/malignant/aug_malignant_14.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/262.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/474.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/98.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/762.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1463.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/332.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1043.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1279.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/894.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/363.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/574.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/274.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/203.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/715.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/7.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/923.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/240.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1076.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/556.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/888.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1211.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1220.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/783.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1307.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1003.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/340.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/34.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/780.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/808.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/149.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/684.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/511.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/885.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/732.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1473.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/344.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/721.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/101.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1069.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1322.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/673.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/925.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1709.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/796.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/970.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1140.jpg', '//kaggle/working/augmented/malignant/aug_malignant_30.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1123.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1780.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/178.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/198.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/1711.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1376.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/1559.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/110.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1486.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1595.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1650.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1433.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/890.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1478.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1223.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1623.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/759.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1032.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/989.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1205.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/909.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/236.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/443.jpg', '//kaggle/working/augmented/malignant/aug_malignant_27.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1154.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1011.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/815.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/353.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1120.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1397.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1542.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/207.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1603.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/201.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/329.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1105.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/17.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/599.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1474.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/336.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1313.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1374.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1196.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/626.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/924.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/913.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/397.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/791.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1179.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/736.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/807.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/392.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1543.jpg', '//kaggle/working/augmented/malignant/aug_malignant_8.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1622.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/300.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1684.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/296.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/990.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/369.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1760.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/272.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/687.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/499.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/934.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1077.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/641.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/740.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/977.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1163.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1118.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/594.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/822.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1475.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/847.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1164.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1117.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1234.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/607.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/120.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/8.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1629.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/187.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1476.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/705.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/482.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1582.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/638.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1217.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/43.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/183.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/119.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/294.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1116.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/241.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1693.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1331.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/264.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/864.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/723.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/192.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/598.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1262.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1041.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/959.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1175.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/713.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1310.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1403.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1311.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/339.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/371.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/808.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1783.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/492.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/863.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/159.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1128.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1285.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/928.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1093.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/68.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/143.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1420.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1206.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1002.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1705.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1059.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/472.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/184.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/478.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/806.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1221.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1451.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1468.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1430.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1532.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1047.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/176.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/969.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/109.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/724.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1399.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/708.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/176.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1795.jpg']

precision,recall,f1,cm=evaluate_model(test_image_urls ,model)
# Display the results
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1 Score: {f1 * 100:.2f}%")
print("Confusion Matrix:")
print(cm)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━

In [309]:
def count_correct_predictions(image_urls):
    correct_predictions = 0
    
    for image_url in image_urls:
        # Extract the actual label from the URL (e.g., 'malignant' or 'benign')
        class_label_from_url = os.path.basename(os.path.dirname(image_url)).lower()
        
        # Get the predicted label
        predicted_label = preprocess_and_predict(image_url,model)
        predicted_label_normalized = predicted_label.lower()
        
        # Compare and count correct predictions
        if predicted_label_normalized == class_label_from_url:
            correct_predictions += 1

    return correct_predictions

# List of image URLs (you can replace this with your actual test image paths)
test_image_urls =['//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/580.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/897.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/1126.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1399.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/171.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1490.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1706.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1707.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/208.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/945.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/187.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/492.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/165.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1426.jpg', '//kaggle/working/augmented/malignant/aug_malignant_189.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/828.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1753.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/723.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1379.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/665.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/365.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1428.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/71.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1270.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/136.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/1628.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1733.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/602.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/643.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/923.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1048.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/959.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1055.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/844.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/195.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/477.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1193.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1635.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/1362.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/1611.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/725.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/546.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/241.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/596.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1301.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1147.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/256.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/631.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/581.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/301.jpg', '//kaggle/working/augmented/malignant/aug_malignant_80.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1697.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1016.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/873.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/230.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1137.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/37.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/132.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/970.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/1118.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/1074.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1531.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/977.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/358.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1439.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/387.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/405.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/816.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/183.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/863.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/742.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/496.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/474.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/617.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1236.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/884.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/683.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/758.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1405.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/654.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/617.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1546.jpg', '//kaggle/working/augmented/malignant/aug_malignant_1.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/392.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1140.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1012.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/731.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/684.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1520.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/929.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/706.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/190.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1434.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1071.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/992.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/429.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/745.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/217.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1026.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/278.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/121.jpg', '//kaggle/working/augmented/malignant/aug_malignant_172.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/272.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1291.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1732.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1556.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/730.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/509.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/622.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/606.jpg', '//kaggle/working/augmented/malignant/aug_malignant_77.jpg', '//kaggle/working/augmented/malignant/aug_malignant_134.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1567.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1529.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1563.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/706.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/698.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/388.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1551.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/236.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/597.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/1151.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/113.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1553.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/473.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/824.jpg', '//kaggle/working/augmented/malignant/aug_malignant_29.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1583.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/357.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/674.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1171.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1605.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1112.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/177.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/461.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1673.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/606.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1652.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/66.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1755.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/868.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1190.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/477.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1179.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1260.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1339.jpg', '//kaggle/working/augmented/malignant/aug_malignant_42.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/512.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/174.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/778.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/829.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/425.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/902.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/30.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/654.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1380.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/299.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1449.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/351.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/244.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/1682.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1103.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/60.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/43.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1287.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/578.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1153.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/711.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1310.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1490.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/843.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1100.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/12.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1001.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/344.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1050.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/297.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1059.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1496.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/65.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/663.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/469.jpg', '//kaggle/working/augmented/malignant/aug_malignant_183.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/806.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1394.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1367.jpg', '//kaggle/working/augmented/malignant/aug_malignant_103.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/332.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/397.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/760.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1061.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1345.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/872.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1206.jpg', '//kaggle/working/augmented/malignant/aug_malignant_242.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/220.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/372.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/237.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/772.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/837.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/201.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/366.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/533.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/623.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/298.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1603.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/738.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/52.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/355.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/1672.jpg', '//kaggle/working/augmented/malignant/aug_malignant_168.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1196.jpg', '//kaggle/working/augmented/malignant/aug_malignant_204.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1009.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/1044.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/744.jpg', '//kaggle/working/augmented/malignant/aug_malignant_67.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1171.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/73.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/122.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/319.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1370.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1076.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1133.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/1745.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1661.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/444.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/385.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/1773.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/142.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/445.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/356.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/683.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/327.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1000.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/212.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/905.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/792.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/613.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/166.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/851.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/763.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/930.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1245.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/379.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/921.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/746.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/497.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/528.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1081.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/374.jpg', '//kaggle/working/augmented/malignant/aug_malignant_108.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/244.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/1441.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/1269.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/673.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1751.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/339.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1158.jpg', '//kaggle/working/augmented/malignant/aug_malignant_243.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/239.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/546.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/1646.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/567.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/5.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1134.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/1409.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/210.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/204.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1029.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/360.jpg', '//kaggle/working/augmented/malignant/aug_malignant_196.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/895.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/537.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/57.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/644.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1040.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1427.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1012.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1508.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/672.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/954.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/686.jpg', '//kaggle/working/augmented/malignant/aug_malignant_151.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1395.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/249.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1096.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1526.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/851.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1527.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/105.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/1139.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1020.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/258.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/235.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/41.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1136.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1150.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1263.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1318.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1787.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/113.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1541.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1591.jpg', '//kaggle/working/augmented/malignant/aug_malignant_201.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1234.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/100.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1749.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1165.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/224.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/1243.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1347.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/491.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/1466.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/620.jpg', '//kaggle/working/augmented/malignant/aug_malignant_250.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/218.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1429.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/815.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/1642.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/898.jpg', '//kaggle/working/augmented/malignant/aug_malignant_199.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1323.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1595.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/608.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/598.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/766.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/9.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1509.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/183.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/701.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/831.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1259.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1690.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/780.jpg', '//kaggle/working/augmented/malignant/aug_malignant_245.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/508.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/689.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/14.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/322.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1095.jpg', '//kaggle/working/augmented/malignant/aug_malignant_62.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1031.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/576.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1069.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1486.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/443.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1340.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/27.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1136.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1669.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1621.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/877.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1275.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/98.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/366.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1589.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1042.jpg', '//kaggle/working/augmented/malignant/aug_malignant_139.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/185.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1039.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/769.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1691.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/32.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/432.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/557.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/448.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/42.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/865.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1333.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1127.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/9.jpg', '//kaggle/working/augmented/malignant/aug_malignant_92.jpg', '//kaggle/working/augmented/malignant/aug_malignant_208.jpg', '//kaggle/working/augmented/malignant/aug_malignant_93.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1228.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1167.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/615.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1790.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/408.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/560.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/862.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1208.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1425.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/737.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/542.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1052.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/705.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1452.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1175.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1770.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1106.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/156.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1522.jpg', '//kaggle/working/augmented/malignant/aug_malignant_65.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1685.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/25.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1339.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/434.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/434.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/47.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/476.jpg', '//kaggle/working/augmented/malignant/aug_malignant_114.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1248.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/213.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1754.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/897.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/390.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1038.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1555.jpg', '//kaggle/working/augmented/malignant/aug_malignant_177.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1313.jpg', '//kaggle/working/augmented/malignant/aug_malignant_38.jpg', '//kaggle/working/augmented/malignant/aug_malignant_200.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1098.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1035.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/650.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1404.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1789.jpg', '//kaggle/working/augmented/malignant/aug_malignant_48.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/728.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1063.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/751.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1363.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1323.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1183.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/922.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/694.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/643.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/977.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1349.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/826.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/371.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1315.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1725.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/354.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/712.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/618.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/704.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1267.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/427.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/229.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1062.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/4.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1250.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1166.jpg', '//kaggle/working/augmented/malignant/aug_malignant_55.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/325.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/261.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1506.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/710.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/771.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/329.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/125.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/48.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/682.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/239.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1154.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/559.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/331.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/749.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/545.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1343.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1123.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1374.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1213.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/157.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/934.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1523.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1681.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1051.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/756.jpg', '//kaggle/working/augmented/malignant/aug_malignant_160.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/3.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/783.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/999.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1346.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/197.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/542.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1499.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/983.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1004.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/534.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/39.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/511.jpg', '//kaggle/working/augmented/malignant/aug_malignant_75.jpg', '//kaggle/working/augmented/malignant/aug_malignant_147.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1665.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/893.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1225.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1419.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1593.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/253.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/350.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1148.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/525.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/948.jpg', '//kaggle/working/augmented/malignant/aug_malignant_132.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/318.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/867.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/21.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/945.jpg', '//kaggle/working/augmented/malignant/aug_malignant_236.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1188.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1132.jpg', '//kaggle/working/augmented/malignant/aug_malignant_5.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/217.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1267.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1072.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/459.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1703.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/15.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1353.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/20.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/240.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1474.jpg', '//kaggle/working/augmented/malignant/aug_malignant_142.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/611.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/464.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1601.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/677.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1194.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/699.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1794.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1533.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/256.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1355.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/16.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/995.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1448.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/912.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/1023.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1482.jpg', '//kaggle/working/augmented/malignant/aug_malignant_135.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1670.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/954.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/38.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/207.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/423.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/31.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/989.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/309.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1370.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/571.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/729.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/826.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1005.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/479.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/971.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1624.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1039.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/380.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/519.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/862.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/257.jpg', '//kaggle/working/augmented/malignant/aug_malignant_17.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/102.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/593.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1073.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1360.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1255.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1564.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/738.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/911.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/985.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1124.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/937.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/883.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1488.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/591.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/64.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/117.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/313.jpg', '//kaggle/working/augmented/malignant/aug_malignant_111.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/82.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/51.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1465.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/23.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1213.jpg', '//kaggle/working/augmented/malignant/aug_malignant_8.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1180.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/394.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/981.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/449.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/493.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1298.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1449.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/471.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1597.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/629.jpg', '//kaggle/working/augmented/malignant/aug_malignant_105.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1472.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/238.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/233.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/674.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1358.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1041.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/1343.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/1080.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/884.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1700.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/205.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/612.jpg', '//kaggle/working/augmented/malignant/aug_malignant_117.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/785.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1106.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/254.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/809.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1189.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/494.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1730.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/225.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/658.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/43.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/513.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/284.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/441.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1216.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1076.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1783.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1303.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/589.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/721.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/564.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/854.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1366.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/193.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/171.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/838.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/349.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1437.jpg', '//kaggle/working/augmented/malignant/aug_malignant_157.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/430.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/482.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/1006.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1020.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1084.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1309.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1406.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/312.jpg', '//kaggle/working/augmented/malignant/aug_malignant_137.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1176.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/213.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/599.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/802.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1128.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/320.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/827.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/223.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/568.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/876.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1288.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/634.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1225.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/352.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/561.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/320.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/514.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1126.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/912.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1148.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1497.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/930.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/464.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1178.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/288.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/97.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/993.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1272.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/758.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/1038.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1258.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1759.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1692.jpg', '//kaggle/working/augmented/malignant/aug_malignant_86.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1369.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/13.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/1256.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/975.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/402.jpg', '//kaggle/working/augmented/malignant/aug_malignant_202.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/271.jpg', '//kaggle/working/augmented/malignant/aug_malignant_166.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/753.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/1070.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/1237.jpg', '//kaggle/working/augmented/malignant/aug_malignant_115.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/282.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/709.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/1244.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/510.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/548.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/481.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/800.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/benign/407.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/malignant/11.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/test/malignant/694.jpg', '//kaggle/input/skin-cancer-malignant-vs-benign/train/benign/483.jpg', '//kaggle/working/augmented/malignant/aug_malignant_211.jpg', '//kaggle/working/augmented/malignant/aug_malignant_21.jpg']
# Count correct predictions
correct_predictions = count_correct_predictions(test_image_urls)
print(f"Number of correct predictions: {correct_predictions}/{len(test_image_urls)}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━